# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [2]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading  ETTh

Load original dataset and preprocess the loaded data.

In [5]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/test_ep/samples/etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../OUTPUT/test_ep/ddpm_fake_test_ep_milestone_10.npy')

ori_data = np.load('../OUTPUT/etth/samples/etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../OUTPUT/etth/ddpm_fake_etth_milestone_20.npy')

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)
b,t,n = ori_data.shape


ori_data = ori_data.transpose(2, 0, 1).reshape(b * n, t, 1)

# fake_data = fake_data[:ori_data.shape[0]*ori_data.shape[2]]
# fake_data = fake_data.reshape(n, b, t).transpose(1, 2, 0)

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)

ori shape is:  (17397, 24, 7)
fake shape is:  (121856, 24, 1)
ori shape is:  (121779, 24, 1)
fake shape is:  (121856, 24, 1)








## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [6]:
for j in range(3):

    context_fid_score = []

    for i in range(iterations):
        context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
        context_fid_score.append(context_fid)
        print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')

    display_scores(context_fid_score)

# Seed 12345 :  mp  10mile  0.292   0.309
# seed 12345 :  ep  10mile  0.151


# mp 24 是0.047， 96 就0.2了应该，但是gpt的24是0.02

#  ep 24 是  0.0127


Iter 0:  context-fid = 0.019706784075510266 

Iter 1:  context-fid = 0.019944352601937332 

Iter 2:  context-fid = 0.01819694524180907 

Iter 3:  context-fid = 0.019691456535924412 

Iter 4:  context-fid = 0.016379515700804922 

Final Score:  0.0187838108311972 ± 0.0018777067487841404
Iter 0:  context-fid = 0.016808809361353923 

Iter 1:  context-fid = 0.02210766592037361 

Iter 2:  context-fid = 0.019119908415471083 

Iter 3:  context-fid = 0.015595383397438439 

Iter 4:  context-fid = 0.019277544653405563 

Final Score:  0.01858186234960852 ± 0.0031207250199783974
Iter 0:  context-fid = 0.020622441044680674 

Iter 1:  context-fid = 0.018915480549050994 

Iter 2:  context-fid = 0.01940250270099083 

Iter 3:  context-fid = 0.01955525506989364 

Iter 4:  context-fid = 0.020750398588037962 

Final Score:  0.019849215590530815 ± 0.0009948500511941108


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [7]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [8]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

# single input    mp 10miles    0.087 + 0.02
# single input    ep 10miles    0.104 + 0.03

Iter 0:  cross-correlation = 8.215650382226159e-16 

Iter 1:  cross-correlation = 2.90878432451791e-15 

Iter 2:  cross-correlation = 6.661338147750939e-17 

Iter 3:  cross-correlation = 6.550315845288424e-16 

Iter 4:  cross-correlation = 1.765254609153999e-15 

Final Score:  1.2434497875801752e-15 ± 1.3819315947113081e-15


## Data Loading Energy

Load original dataset and preprocess the loaded data.

In [11]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/test_ep/samples/etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../OUTPUT/test_ep/ddpm_fake_test_ep_milestone_10.npy')

ori_data = np.load('../OUTPUT/energy/samples/energy_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../OUTPUT/energy/ddpm_fake_energy_milestone_20.npy')

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)
b,t,n = ori_data.shape


ori_data = ori_data.transpose(2, 0, 1).reshape(b * n, t, 1)

# fake_data = fake_data[:ori_data.shape[0]*ori_data.shape[2]]
# fake_data = fake_data.reshape(n, b, t).transpose(1, 2, 0)

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)

ori shape is:  (19712, 24, 28)
fake shape is:  (552160, 24, 1)
ori shape is:  (551936, 24, 1)
fake shape is:  (552160, 24, 1)








## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [12]:
for j in range(3):

    context_fid_score = []

    for i in range(iterations):
        context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
        context_fid_score.append(context_fid)
        print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')

    display_scores(context_fid_score)

# Seed 12345 :  mp  10mile  0.292   0.309
# seed 12345 :  ep  10mile  0.151


# mp 24 是0.047， 96 就0.2了应该，但是gpt的24是0.02

#  ep 24 是  


Iter 0:  context-fid = 0.03313546176615627 

Iter 1:  context-fid = 0.033700654755642094 

Iter 2:  context-fid = 0.03686725740685956 

Iter 3:  context-fid = 0.03337610062454661 

Iter 4:  context-fid = 0.03404903406211731 

Final Score:  0.03422570172306437 ± 0.0018825800024332103
Iter 0:  context-fid = 0.03405157638952414 

Iter 1:  context-fid = 0.02911736382729563 

Iter 2:  context-fid = 0.030531896600345776 

Iter 3:  context-fid = 0.03259609961619503 

Iter 4:  context-fid = 0.03186650565312128 

Final Score:  0.03163268841729637 ± 0.00235363716350389
Iter 0:  context-fid = 0.03847710655623514 

Iter 1:  context-fid = 0.03196264700462169 

Iter 2:  context-fid = 0.03800480703789828 

Iter 3:  context-fid = 0.0377112927799391 

Iter 4:  context-fid = 0.035297786432300866 

Final Score:  0.03629072796219902 ± 0.003369381750713639


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [13]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [14]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

# single input    mp 10miles    0.087 + 0.02
# single input    ep 10miles    0.104 + 0.03

Iter 0:  cross-correlation = 6.106226635438361e-16 

Iter 1:  cross-correlation = 2.55351295663786e-16 

Iter 2:  cross-correlation = 2.7755575615628914e-15 

Iter 3:  cross-correlation = 1.042499420123022e-14 

Iter 4:  cross-correlation = 1.3655743202889425e-14 

Final Score:  5.5444537849780315e-15 ± 7.593381226140188e-15
